# Moving Beyond Linear Models

### Polynomial Regression
Add additional predictors that are the original predictors raised to a power, ie X, X^2, X^3

### Step Functions
Cut the range of a variable into K distinct regions to produce a qualitative variable.  Fits like a piecewise constant function.

### Regression Splines
Similar to Step Function & Poly Regression, divide range of X into distinct regions and fit each region using a polynomial function.  But, they are constrained so they join smoothly to each region boundary (aka knot)

### Smoothing Splines
Similar to regression splines, but result from minimizing RSS with a smoothness penalty.

### Local Regression
Similar to splines, but regions can overlap, in a smooth way

### Generalized Additive Model
Extend above methods to work with multiple predictors





### 6.


In [14]:
require(ISLR)
require(boot)
attach(Wage)

Loading required package: boot
The following objects are masked from Wage (pos = 4):

    age, education, health, health_ins, jobclass, logwage, maritl,
    race, region, wage, year

The following objects are masked from Wage (pos = 5):

    age, education, health, health_ins, jobclass, logwage, maritl,
    race, region, wage, year

The following objects are masked from Wage (pos = 6):

    age, education, health, health_ins, jobclass, logwage, maritl,
    race, region, wage, year

The following objects are masked from Wage (pos = 7):

    age, education, health, health_ins, jobclass, logwage, maritl,
    race, region, wage, year



In [23]:
cv.error <- rep(0,10)
for (i in 1:10) {
    glm.fit <- glm(wage~poly(age,i), data=Wage)
    cv.error[i] <- cv.glm(Wage, glm.fit, K=10)$delta[1]  # [1]:std, [2]:bias-corrected
    print(cv.glm(Wage, glm.fit, K=10))

}
cv.error
?cv.glm


$call
cv.glm(data = Wage, glmfit = glm.fit, K = 10)

$K
[1] 10

$delta
[1] 1676.657 1676.520

$seed
  [1]         403          49 -1920910413 -1766702313  1937170047  -431580769
  [7] -1203829625  -133511518 -2045948678 -1972481626   -49050547  1957535537
 [13] -1290100577 -1080213187   810279452 -1078344427  -810172655 -1017444657
 [19]   702697649   204921916    44169601 -1861767263 -1520838863   900419343
 [25]   484715774  1435554288  -437809487   503234744  -242689753 -1284667912
 [31]  -252091049   296958949   238300010   396396358  1398776638   -54674291
 [37]  1021705528  1915438378   990159551  1051242473 -2094585016   322864528
 [43] -2140772461 -1671143845   732320777  1927817730  -555526456  -579147816
 [49]  1270736757  1922185294 -1142995772 -2040016750   691290651 -1048607340
 [55] -1054061605  1885057924 -1505395526   863234793   953300667  1638484028
 [61]   554278752 -1930687523  1631036276 -1031956647  1733814621   761362901
 [67] -1472054450 -1797813577 -1483209556 

[1] 1676.383 1600.524 1597.791 1594.572 1594.089 1594.748 1592.679 1596.980
 [9] 1596.189 1593.530